In [ ]:
import numpy as np
import matplotlib.pyplot as plt

P = 2**254 + 17 * 2**241 + 1  # A prime field
SQRT_P = int(P**0.5)
DENOMINATOR = 1125899906842624000  # Equivalent to 2^50 * 1000

def mod_inverse(a, p):
    return pow(a, p - 2, p)

def finite_field_add(a, b):
    return (a + b) % P

def finite_field_sub(a, b):
    return (a - b) % P

def finite_field_mul(a, b):
    return (a * b) % P

def finite_field_div(a, b):
    return finite_field_mul(a, mod_inverse(b, P))

def finite_field_mod(a, b):
    return a % b

def finite_field_dot(a, b, denominator):
    return finite_field_div(finite_field_add(finite_field_mul(a[0], b[0]), finite_field_mul(a[1], b[1])), denominator)

def random_integer_in_range(seed):
    np.random.seed(seed)
    return np.random.randint(0, 16)

def generate_random_unit_vector(seed):
    vecs = [[1000, 0], [923, 382], [707, 707], [382, 923], [0, 1000], [-383, 923], [-708, 707], [-924, 382],
            [-1000, 0], [-924, -383], [-708, -708], [-383, -924], [-1, -1000], [382, -924], [707, -708], [923, -383]]
    idx = random_integer_in_range(seed)
    return vecs[idx]

def get_corners_and_grad_vectors(p, scale, key):
    xmodulo_remainder = finite_field_mod(p[0], scale)
    ymodulo_remainder = finite_field_mod(p[1], scale)

    bottom_left = [finite_field_sub(p[0], xmodulo_remainder), finite_field_sub(p[1], ymodulo_remainder)]
    bottom_right = [finite_field_add(bottom_left[0], scale), bottom_left[1]]
    top_left = [bottom_left[0], finite_field_add(bottom_left[1], scale)]
    top_right = [finite_field_add(bottom_left[0], scale), finite_field_add(bottom_left[1], scale)]

    corners = [bottom_left, bottom_right, top_left, top_right]

    grad_vectors = [generate_random_unit_vector(key + i) for i in range(4)]

    return corners, grad_vectors

def get_weight(corner, p, denominator):
    diff = [finite_field_sub(p[0], corner[0]), finite_field_sub(p[1], corner[1])]
    factor = [finite_field_sub(denominator, diff[0]), finite_field_sub(denominator, diff[1])]
    nominator = finite_field_mul(factor[0], factor[1])
    return finite_field_div(nominator, denominator)

def perlin_value(grads, coords, scale, p, denominator):
    weights = [get_weight(coords[i], p, denominator) for i in range(4)]
    dist_vec = [[finite_field_sub(p[0], coords[i][0]), finite_field_sub(p[1], coords[i][1])] for i in range(4)]
    scaled_dist_vec = [[finite_field_div(dist_vec[i][0], scale), finite_field_div(dist_vec[i][1], scale)] for i in range(4)]

    ret = [finite_field_div(finite_field_mul(weights[i], finite_field_dot(grads[i], scaled_dist_vec[i], denominator)), denominator) for i in range(4)]
    return sum(ret) % P

def single_scale_perlin(p, key, scale, denominator, sqrt_p):
    corners, grads = get_corners_and_grad_vectors(p, scale, key)
    p_scaled = [finite_field_mul(denominator, p[0]), finite_field_mul(denominator, p[1])]
    coords_scaled = [[finite_field_mul(denominator, corner[0]), finite_field_mul(denominator, corner[1])] for corner in corners]
    return perlin_value(grads, coords_scaled, scale, p_scaled, denominator)

def multi_scale_perlin(p, key, scale, x_mirror, y_mirror):
    perlin_results = [single_scale_perlin(p, key, scale * 2**i, DENOMINATOR, SQRT_P) for i in range(3)]
    perlin_sum = sum(perlin_results) % P
    return finite_field_div(perlin_sum, 4)

# Example usage to generate Perlin noise over a grid
key = 42
scale = 4
x_mirror = 0
y_mirror = 0

grid_size = 100
noise_grid = np.zeros((grid_size, grid_size))

for i in range(grid_size):
    for j in range(grid_size):
        p = [i, j]
        noise_grid[i, j] = multi_scale_perlin(p, key, scale, x_mirror, y_mirror)

# Plotting the Perlin noise
plt.figure(figsize=(10, 10))
plt.imshow(noise_grid, cmap='gray', interpolation='nearest')
plt.title('Perlin Noise')
plt.colorbar()
plt.show()


: 